In [2]:
import json
from typing import List, Dict, Tuple
import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


/root/miniconda3/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-07-15 20:28:54.228250: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-15 20:28:54.282386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-15 20:28:55.327348: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38

#### 预处理

将数据集处理成以下json格式

```json
[
  {
    'question': '问题',
    'answer': '答案'
  }
]
```

##### 读取全部类别的问答并保存

In [ ]:
data = []
with open('QA_source.json', 'r', encoding='utf-8') as f:
    for line in f:
        line = json.loads(line)
        question: str = line['question']
        answers: List[str] = line['answer']
        while len(answers) > 0:
            data.append({
                'question': line['question'],
                'answer': answers[0]
            })
            answers.pop(0)

with open('QA.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

print('done for preparing datasets of all categories')

##### 只保存某类别的问答

###### 类别

```json
{'婚姻家庭': 39281, '劳动纠纷': 35011, '交通事故': 22646, '债权债务': 21925, '刑事辩护': 18314, '合同纠纷': 13765, '房产纠纷': 12071, '侵权': 10594, '公司法': 10011, '医疗纠纷': 7285, '拆迁安置': 7022, '行政诉讼': 2776, '建设工程': 1610, '知识产权': 530, '综合咨询': 176, '人身损害': 98, '涉外法律': 69, '海事海商': 46, '消费权益': 27, '抵押担保': 26, '电信通讯': 18, '土地纠纷': 16, '离婚': 16, '工伤赔偿': 15, '继承': 13, '保险理赔': 12, '银行': 12, '网络法律': 9, '求学教育': 8, '税务': 6, '取保候审': 6, '行政复议': 5, '倾销补贴': 3, '毒品犯罪': 3, '公安国安': 3, '调解谈判': 3, '经销代理': 3, '合伙联营': 3, '股权纠纷': 2, '招商引资': 2, '票据': 2, '行政': 2, '资产拍卖': 2, '旅游': 2, '期货交易': 1, '水利电力': 1, '法律文书代写': 1, '工商查询': 1, '兼并收购': 1, '海关商检': 1, '金融证券': 1, '污染损害': 1, '广告宣传': 1, '刑事自诉': 1}
```

###### 处理

In [12]:
data = []
category = '公司法'

with open('QA_source.json', 'r', encoding='utf-8') as f:
    for line in f:
        line = json.loads(line)
        if line['category'] != category:
            continue

        question: str = line['question']
        answers: List[str] = line['answers']
        while len(answers) > 0:
            data.append({
                'question': question.strip(),
                'answer': answers[0].strip()
            })
            answers.pop(0)

with open(f'QA_{category}.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

print(f'done for preparing datasets of {category} categories')

done for preparing datasets of 公司法 categories


#### 实现一个中文检索器

该检索器包含：
1. sparse_retrieve: BM25稀疏检索
2. dense_retrieve: BERT稠密检索
3. hybrid_retrieve: 混合检索

一般的 Retriever 流程如下：

1. 文本预处理: 分词(英文-空格，中文-单个字符)，清洗(去噪-HTML标签/特殊字符)
2. 文本嵌入: 将文本转换为固定维度的向量
3. 检索算法:
   - 倒排索引(如 BM25)
   - 向量检索(如 向量数据库FAISS/Milvus, cos-sim)
   - 混合检索(结合向量检索和倒排索引, 初筛+精筛/综合得分)
4. 结果重排序
   - **相似度**：向量相似度
   - **模型**：排序模型（如 BERT-based ranking model）对候选文档进行排序
   - **混合排序**：多种排序方法 (如先按向量相似度排序，再按关键词匹配度排序)

In [3]:
class ChineseRetriever:
    def __init__(self,
                 json_path: str,
                 model_name: str = 'bert-base-chinese'
                 ):
        """
        初始化中文检索器
        :param json_path: JSON文件路径，格式为[{'question':'', 'answer':''}, ...]
        :param model_name: 稠密检索模型名称 or 路径(默认bert-base-chinese)
        """
        self.docs = self._load_json_docs(json_path)
        self.questions = [doc['question'] for doc in self.docs]
        self.answers = [doc['answer'] for doc in self.docs]

        # 初始化稀疏检索(BM25)
        self.bm25 = self._init_sparse_retriever()

        # 初始化稠密检索(BERT)
        self.encoder = SentenceTransformer(model_name)
        self.doc_embeddings = self._init_dense_retriever()

    def _load_json_docs(self, path: str) -> List[Dict]:
        """加载并预处理JSON文档"""
        with open(path, 'r', encoding='utf-8') as f:
            docs = json.load(f)

        # 中文分词处理(简单按字符分割)
        for doc in docs:
            # 后续可使用jieba分词
            doc['tokenized'] = list(doc['question'])  # 按字符级分词, 即一个中文字为一个token
        return docs

    def _init_sparse_retriever(self):
        """初始化BM25稀疏检索器"""
        tokenized_corpus = [doc['tokenized'] for doc in self.docs]
        return BM25Okapi(tokenized_corpus)

    def _init_dense_retriever(self) -> np.ndarray:
        """预计算所有文档的BERT嵌入"""
        return self.encoder.encode(self.questions)

    def sparse_retrieve(self, query: str, top_k: int = 5) -> List[Tuple[int, float]]:
        """
        BM25稀疏检索
        :return: 返回(top_k个(文档索引, 得分))列表，如[(3, 8.21), (1, 6.54)...]，按得分降序排序
        :param query: 原始查询字符串
        :process: 
            1. 先将中文字符串转换为字符列表 (即进行文本分割), BM25基于词频统计，中文无空格因此需主动分词，目前仅使用字符级分词，后续可尝试分词工具，如 jieba
            2. 再计算BM25得分
            3. 最后返回前k个文档
        """
        tokenized_query = list(query)  # 中文按字符分词
        # BM25计算公式
        scores = self.bm25.get_scores(tokenized_query)
        # 前k个文档的编号
        top_indices = np.argsort(scores)[-top_k:][::-1]
        return [(i, scores[i]) for i in top_indices]

    def dense_retrieve(self, query: str, top_k: int = 5) -> List[Tuple[int, float]]:
        """
        BERT稠密检索
        :return: 返回(top_k个(文档索引, 余弦相似度))列表
        :param query: 原始查询字符串
        :process: 
            1. 分词器将文本转换为子词单元
            2. 添加[CLS]/[SEP]等特殊标记
            3. 通过12层Transformer获取[CLS]位置的768维向量
        """
        # 编码
        query_embedding = self.encoder.encode(query)
        # 余弦相似度
        similarities = cosine_similarity(
            [query_embedding],
            self.doc_embeddings
        )[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        return [(i, similarities[i]) for i in top_indices]

    def hybrid_retrieve(self, query: str, top_k: int = 5,
                        dense_weight: float = 0.7) -> List[Tuple[int, float]]:
        """
        混合检索(默认稠密权重0.7，稀疏权重0.3)
        :param dense_weight: 稠密检索得分权重
        :return: 返回(top_k个(文档索引, 综合得分))列表
        """
        # 各自检索2倍于最终需要的文档量，扩大候选池
        sparse_results = dict(self.sparse_retrieve(query, top_k * 2))
        dense_results = dict(self.dense_retrieve(query, top_k * 2))

        # 归一化得分
        # BM25得分范围[0, +∞]
        max_sparse = max(sparse_results.values()) if sparse_results else 1
        # 余弦相似度[-1, 1]
        max_dense = max(dense_results.values()) if dense_results else 1

        # 融合得分
        fused_scores = {}
        all_indices = set(sparse_results.keys()) | set(dense_results.keys())
        for idx in all_indices:
            norm_sparse = sparse_results.get(idx, 0) / max_sparse
            norm_dense = dense_results.get(idx, 0) / max_dense
            fused_scores[idx] = dense_weight * norm_dense + (1 - dense_weight) * norm_sparse

        # 返回top_k结果
        top_indices = sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
        return top_indices

    def get_document(self, index: int) -> Dict:
        """根据索引获取完整文档"""
        return self.docs[index]

    def print_results(self, results: List[Tuple[int, float]]):
        """格式化打印检索结果"""
        for rank, (idx, score) in enumerate(results, 1):
            doc = self.get_document(idx)
            print(f"Rank {rank} (Score: {score:.4f}):")
            print(f"Q: {doc['question']}")
            print(f"A: {doc['answer']}\n")


In [9]:
# 初始化检索器
retriever = ChineseRetriever("QA_公司法.json", model_name='/root/autodl-tmp/data/models/bert-base-chinese')

No sentence-transformers model found with name /root/autodl-tmp/data/models/bert-base-chinese. Creating a new one with mean pooling.


In [10]:
# 测试查询
query = "注册公司有哪些注意事项"

In [11]:
print("=" * 40 + "\n稀疏检索结果(BM25):")
sparse_results = retriever.sparse_retrieve(query)
retriever.print_results(sparse_results)


稀疏检索结果(BM25):
Rank 1 (Score: 35.0303):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 6、一般纳税人资格

Rank 2 (Score: 35.0303):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 企业按组建形式可以分为有限公司、个人独资企业、合伙企业。目前，90%以上的企业类型为有限公司(以注册资本承担对外赔偿限额)，而个人独资企业或合伙企业因投资者承担无限责任而选择这2种企业类型的较少。

Rank 3 (Score: 35.0303):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 费用由行政收费、银行开户费用、验资费及代理公司服务费构成。但是，各个区及开发区对于公司注册登记费用的补贴政策是不一样的。

Rank 4 (Score: 35.0303):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 2、要求

Rank 5 (Score: 35.0303):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 5、进出口权


In [12]:
print("=" * 40 + "\n稠密检索结果(BERT):")
dense_results = retriever.dense_retrieve(query)
retriever.print_results(dense_results)


稠密检索结果(BERT):
Rank 1 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 5、进出口权

Rank 2 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 按照《公司法》的规定，有限公司最低注册资本为3万元人民币，其中，一人有限公司最低注册资本为10万元人民币。注册资本可以分期出资，首批不低于20%，其余注册资本可在2年内到位。但是，不同对于最低注册资本的要求是不一样的。例如，国际货运代理公司要求最低注册资本为500元人民币。需注意的是，任何公司在设立登记时，除了要符合公司法对注册资本的要求，也要符合行业法规对最低注册资本的规定。在还需符合外资企业法律法规对于注册资本的要求

Rank 3 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 首先，办理公司注册登记，需特别注意的是在公司经营范围中需加上“从事货物及技术的进出口业务”这一条。有了这条经营范围就才能申请进出口备案。从事进出口业务的也写清楚具体的业务范围。其次，在公司注册完毕及银行开户之后，申请进出口备案。进出口备案包括海关、电子口岸、外汇、检验检疫等备案手续。

Rank 4 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 对于贸易公司或进出口公司来说，基本上都要申请一般纳税人资格(开具增值税专用发票)。各个区或同一个区的不同税务所对于企业申请一般纳税人资格的要求或规定是有些差异的

Rank 5 (Score: 0.9319):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 2、要求


In [13]:
print("=" * 40 + "\n混合检索结果:")
hybrid_results = retriever.hybrid_retrieve(query)
retriever.print_results(hybrid_results)


混合检索结果:
Rank 1 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 首先，办理公司注册登记，需特别注意的是在公司经营范围中需加上“从事货物及技术的进出口业务”这一条。有了这条经营范围就才能申请进出口备案。从事进出口业务的也写清楚具体的业务范围。其次，在公司注册完毕及银行开户之后，申请进出口备案。进出口备案包括海关、电子口岸、外汇、检验检疫等备案手续。

Rank 2 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 5、进出口权

Rank 3 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 6、一般纳税人资格

Rank 4 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 按照《公司法》的规定，有限公司最低注册资本为3万元人民币，其中，一人有限公司最低注册资本为10万元人民币。注册资本可以分期出资，首批不低于20%，其余注册资本可在2年内到位。但是，不同对于最低注册资本的要求是不一样的。例如，国际货运代理公司要求最低注册资本为500元人民币。需注意的是，任何公司在设立登记时，除了要符合公司法对注册资本的要求，也要符合行业法规对最低注册资本的规定。在还需符合外资企业法律法规对于注册资本的要求

Rank 5 (Score: 1.0000):
Q: 您好！请问注册公司需要多长时间。有哪些注意事项
A: 4、特殊项目审批
